# R2 Overview:

Remember to do:

Setup

List functions, then print code for one

In [145]:
import pexpect
import sys
import time

from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

rp = pexpect.spawnu('r2 -d func-pointer', encoding='utf-8')

@register_line_magic
def r(line):
    rp.sendline(line)
    time.sleep(.5)
    end = ""
    while not "]>" in end:
        read = rp.read_nonblocking(100000)
        print(read)
        end = read[-50:]

@register_line_magic
def ro():
    print(rp.read_nonblocking(100000))

@register_line_magic
def ri(line):
    rp.sendline(line)
    time.sleep(.5)
    print(rp.read_nonblocking(100000))

In [141]:
%r aa;e asm.syntax=att; afl;

Process with PID 3969 started...
= attach 3969 3969
bin.baddr 0x00400000
Using 0x400000
asm.bits 64
[x] Analyze all flags starting with sym. and entry0 (aa)
0x00400490    1 42           entry0
0x004004c0    4 42   -> 37   sym.deregister_tm_clones
0x004004f0    4 58   -> 55   sym.register_tm_clones
0x00400530    3 34   -> 29   entry.fini0
0x00400560    1 7            entry.init0
0x00400660    1 2            sym.__libc_csu_fini
0x00400664    1 9            sym._fini
0x00400593    1 22           sym.other
0x00400470    1 6            sym.imp.printf
0x004005f0    4 101          sym.__libc_csu_init
0x00400567    1 27           sym.special
0x00400460    1 6            sym.imp.puts
0x004005a9    1 58           main
0x00400480    1 6            sym.imp.gets
0x00400582    1 17           sym.normal
0x00400438    3 23           sym._init
[0x7f316b6ab090]>  len 3722


In [142]:
%r pdf @main

            ; DATA XREF from entry0 @ 0x4004ad
┌ 58: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_30h @ rbp-0x30
│           ; var int64_t var_24h @ rbp-0x24
│           ; var int64_t var_16h @ rbp-0x16
│           ; var int64_t var_8h @ rbp-0x8
│           ; arg int argc @ rdi
│           ; arg char **argv @ rsi
│           0x004005a9      55             pushq %rbp
│           0x004005aa      4889e5         movq %rsp, %rbp
│           0x004005ad      4883ec30       subq $0x30, %rsp
│           0x004005b1      897ddc         movl %edi, var_24h          ; argc
│           0x004005b4      488975d0       movq %rsi, var_30h          ; argv
│           0x004005b8      48c745f88205.  movq $sym.normal, var_8h    ; 0x400582
│           0x004005c0      488d45ea       leaq var_16h, %rax
│           0x004005c4      4889c7         movq %rax, %rdi
│           0x004005c7      b800000000     movl $0, %eax
│           0x004005cc      e8affeffff     callq sym.imp.gets   

# R2: Set Break Points:


In [10]:
%r ds

ds




In [143]:
%r db 0x004005d5

[0x7f316b6ab090]>  len 1263


In [144]:
%r dc

 len 1637f316b6ab090]> dc


# R2 Interact:

In [146]:
%ri aaaaaa

aaaaaa
hit breakpoint at: 4005d5
[0x004005d5]> 


In [154]:
%r pdf

            ; DATA XREF from entry0 @ 0x4004ad
┌ 58: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_30h @ rbp-0x30
│           ; var int64_t var_24h @ rbp-0x24
│           ; var int64_t var_16h @ rbp-0x16
│           ; var int64_t var_8h @ rbp-0x8
│           ; arg int argc @ rdi
│           ; arg char **argv @ rsi
│           0x004005a9      55             pushq %rbp
│           0x004005aa      4889e5         movq %rsp, %rbp
│           0x004005ad      4883ec30       subq $0x30, %rsp
│           0x004005b1      897ddc         movl %edi, var_24h          ; argc
│           0x004005b4      488975d0       movq %rsi, var_30h          ; argv
│           0x004005b8      48c745f88205.  movq $sym.normal, var_8h    ; rdx
│                                                                      ; 0x400582
│           0x004005c0      488d45ea       leaq var_16h, %rax
│           0x004005c4      4889c7         movq %rax, %rdi
│           0x004005c7      b800000000     m

In [157]:
%r px @ rbp-0x8

- offset -       0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x7ffc454060c8  8205 4000 0000 0000 f005 4000 0000 0000  ..@.......@.....
0x7ffc454060d8  ca2c 4f6b 317f 0000 c861 4045 fc7f 0000  .,Ok1....a@E....
0x7ffc454060e8  0000 0000 0100 0000 a905 4000 0000 0000  ..........@.....
0x7ffc454060f8  d927 4f6b 317f 0000 0000 0000 0000 0000  .'Ok1...........
0x7ffc45406108  939f 795c 993b 75da 9004 4000 0000 0000  ..y\.;u...@.....
0x7ffc45406118  0000 0000 0000 0000 0000 0000 0000 0000  ................
0x7ffc45406128  0000 0000 0000 0000 939f 5996 99b1 8d25  ..........Y....%
0x7ffc45406138  939f 9f0e 87ed 1724 0000 0000 0000 0000  .......$........
0x7ffc45406148  0000 0000 0000 0000 0000 0000 0000 0000  ................
0x7ffc45406158  0100 0000 0000 0000 c861 4045 fc7f 0000  .........a@E....
0x7ffc45406168  d861 4045 fc7f 0000 8061 6d6b 317f 0000  .a@E.....amk1...
0x7ffc45406178  0000 0000 0000 0000 0000 0000 0000 0000  ................
0x7ffc45406188  9004 4000 0000 0000 c0

In [ ]:
%r pdf @sym.normal

# Other Notes: